In [7]:
import sys
sys.path.append('/home/pranavsatheesh/host_galaxies/')
import illustris_python as il
import matplotlib.pyplot as plt
import h5py
import numpy as np
from scipy.spatial import cKDTree

In [3]:
basepath = '/orange/lblecha/IllustrisTNG/Runs/TNG50-1/output'
merger_file_path = '/home/pranavsatheesh/host_galaxies/data/merger_files'

In [140]:
merger_file_1bh = merger_file_path+'/galaxy-mergers_TNG50-1_gas-100_dm-100_star-100_bh-001.hdf5'

In [141]:
fmergers = h5py.File(merger_file_1bh, 'r')

In [154]:
fmergers.attrs['box_volume_mpc']**(1/3) * 1000

51668.14289932093

In [150]:
fmergers.attrs['box_volume_mpc']/10**3

137.93311879725817

In [148]:
for attr_name in fmergers.attrs:
    print(f"{attr_name}: {fmergers.attrs[attr_name]}")

HubbleParam: 0.6774
Omega0: 0.3089
OmegaLambda: 0.6911
box_volume_mpc: 137933.11879725818
created: 2024-12-18 01:35:27.457083
merger_components_in_data_arrays: (FirstProg, NextProg, Descendant)
min_parts: [100 100 100   1]
num_mergers: 467
part_names: ['gas' 'dm' 'star' 'bh']
part_types: [0 1 4 5]
snaptimes: [0.04751386 0.06254232 0.07704034 0.08350282 0.09093728 0.09625777
 0.09997661 0.10582597 0.11096103 0.11634526 0.12141409 0.12491542
 0.13348287 0.14263794 0.1460576  0.15314485 0.16057599 0.1667797
 0.17653754 0.18422877 0.19316822 0.1996827  0.21236949 0.22267443
 0.2334794  0.24949284 0.25668768 0.26787083 0.2795412  0.29034102
 0.30155808 0.31172794 0.32224077 0.33310814 0.344342   0.35427208
 0.36448853 0.37499959 0.38398999 0.39506344 0.40071935 0.41423335
 0.42416436 0.43433346 0.44474635 0.45325613 0.46412269 0.47300321
 0.48205365 0.49127726 0.50067736 0.51268081 0.52002058 0.52997065
 0.54011111 0.55044559 0.55832602 0.56900902 0.57715521 0.5881985
 0.59661941 0.60803512

In [135]:
snap = 50
subhalos = il.groupcat.loadSubhalos(
                basepath, snap, 
                fields=['SubhaloLenType', 'SubhaloMassType', 'SubhaloBHMass', 'SubhaloBHMdot', 'SubhaloSFR','SubhaloPos','SubhaloHalfmassRadType']
            )

header = il.groupcat.loadHeader(basepath, snap)
h =header['HubbleParam']
box_size = header['BoxSize']/h
z = header['Redshift']
print(header['Redshift'])
subhalo_pos = subhalos['SubhaloPos']/h 

0.9972942257819404


In [151]:
box_size

51668.142899320934

In [137]:
subhalo_ids = np.arange(0,len(subhalos['SubhaloPos']))
print(len(subhalo_ids))

6780233


In [138]:
Ngas = subhalos['SubhaloLenType'][:, 0]
Ndm = subhalos['SubhaloLenType'][:, 1]
Nstar = subhalos['SubhaloLenType'][:, 4]
Nbh = subhalos['SubhaloLenType'][:, 5]

valid_mask = (Ngas >= 100) & (Ndm >= 100) & (Nstar >= 100) & (Nbh >= 0)
print(valid_mask)
subhalo_ids = subhalo_ids[valid_mask]
subhalo_pos = subhalo_pos[valid_mask]
print(len(subhalo_ids))

[ True  True  True ... False False False]
20163


In [139]:
tree = cKDTree(subhalo_pos, boxsize=box_size)
distances, nearest_neighbors = tree.query(subhalo_pos,k=2)

nearest_subhalo_ids = nearest_neighbors[:,-1]
r_subhalos = distances[:,-1]

stellar_half_mass_radius = subhalos['SubhaloHalfmassRadType'][:,4]/h


r_sep = r_subhalos/(stellar_half_mass_radius[subhalo_ids] + stellar_half_mass_radius[nearest_subhalo_ids] )


valid_mask = r_sep>2 #only including non interacting galaxies with a closest companinon relative seperation above 2

subhalo_ids = subhalo_ids[valid_mask]
print(len(subhalo_ids))

20056
